In [1]:
from RobinRollandModel.datautils import TipGenerator, visualize
from RobinRollandModel.main import RRModel
import RobinRollandModel.pyironjob
from pyiron import Project
import matplotlib.pyplot as plt

/opt/homebrew/anaconda3/envs/pyiron_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pr = Project("Example")

In [ ]:
structure = pr.create.structure.bulk('Al','fcc',4.01,cubic=True)

In [ ]:
visualize(structure)

In [ ]:
tip = TipGenerator(structure,h=120,ah=80,zheight=50)

In [ ]:
pr_tip = tip.create_tip_pyiron(pr)

In [ ]:
visualize(pr_tip)

In [ ]:
tip_shank = TipGenerator(structure=structure,h=120,ah=80,alpha=5,zheight=100)

In [ ]:
pr_tip_shank = tip_shank.create_tip_pyiron(pr)

In [ ]:
visualize(pr_tip_shank)

In [ ]:
al_apt_sim = RRModel(tip_generator=tip,structure=pr_tip,e_field=4)

In [ ]:
tip_output_al = al_apt_sim.charge_distribution_z(structure=pr_tip,e_field=4,radius=20,steps=1000,epsilon=1e-9)

In [ ]:
visualize(pr_tip,charge=tip_output_al['final_charge'],surf_indices=tip_output_al['surface_indices'])

In [ ]:
al_apt_shank = RRModel(tip_generator=tip_shank,structure=pr_tip_shank,e_field=4)

In [ ]:
tip_output_al_shank = al_apt_shank.charge_distribution_z(structure=pr_tip_shank,e_field=4,radius=20,steps=1000,epsilon=1e-9)

In [ ]:
visualize(pr_tip_shank,charge=tip_output_al_shank['final_charge'],surf_indices=tip_output_al_shank['surface_indices'])

In [ ]:
al_apt_sim.run_evaporation()

# pyiron implementation

In [2]:
pr = Project("Example_pyiron")

In [3]:
test = pr.create.job.RRModelAPTjob('test',delete_existing_job=True)

In [4]:
structure = pr.create.structure.bulk('Al','fcc',4.01,cubic=True)

In [5]:
test.input.basic_structure = structure
test.input.e_field = 4
test.input.tip_height = 80
test.input.tip_radius = 20
test.input.z_height = 50
test.input.num_atoms = 10

In [6]:
tipg = test.create_input_structure()

In [7]:
test.input.structure.plot3d(scalar_field=test.input.structure.positions[:,2],mode='plotly')

In [8]:
test.run()

The job test was saved and received the ID: 1


/Users/shyamkatnagallu/Projects/APTrajectories/RobinRollandModel/main.py:64: RuntimeWarning:

invalid value encountered in divide



In [9]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,None,test,/test,None,/Users/shyamkatnagallu/Projects/APTrajectories/Example_pyiron/,2024-02-09 13:28:20.186343,None,None,pyiron@shyams-mbp-2.mpie.de#1,RRModelAPTjob,0.4,None,None


In [10]:
test.output.evaporation_trajectories

{0.0: array([[ 2.00500000e+01,  8.02000000e+00,  7.41850000e+01],
        [ 2.00500174e+01,  8.01921428e+00,  7.41861061e+01],
        [ 2.00500435e+01,  8.01803570e+00,  7.41877652e+01],
        [ 2.00500782e+01,  8.01646413e+00,  7.41899775e+01],
        [ 2.00501217e+01,  8.01449938e+00,  7.41927433e+01],
        [ 2.00501738e+01,  8.01214120e+00,  7.41960628e+01],
        [ 2.00502347e+01,  8.00938927e+00,  7.41999364e+01],
        [ 2.00503043e+01,  8.00624321e+00,  7.42043646e+01],
        [ 2.00503827e+01,  8.00270258e+00,  7.42093480e+01],
        [ 2.00504698e+01,  7.99876687e+00,  7.42148871e+01],
        [ 2.00505656e+01,  7.99443553e+00,  7.42209826e+01],
        [ 2.00506703e+01,  7.98970793e+00,  7.42276353e+01],
        [ 2.00507837e+01,  7.98458340e+00,  7.42348460e+01],
        [ 2.00509060e+01,  7.97906119e+00,  7.42426157e+01],
        [ 2.00510371e+01,  7.97314051e+00,  7.42509451e+01],
        [ 2.00511770e+01,  7.96682051e+00,  7.42598355e+01],
        [ 2.0051325

In [ ]:
from cProfile import Profile
with Profile() as prof:
    tip = TipGenerator(structure,h=80,ah=20,zheight=50)
    pr_tip = tip.create_tip_pyiron(pr)
    al_apt_sim = RRModel(tip_generator=tip,structure=pr_tip,e_field=4)
    prof.dump_stats('my_apt_imp.pyprofile')
    prof.print_stats()

In [ ]:
import pstats
stream = open('my_apt_imp.txt', 'w')
stats = pstats.Stats('my_apt_imp.pyprofile', stream=stream)
stats.sort_stats('tottime')
stats.print_stats()